<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [8]</a>'.</span>

In [1]:
params = {
    "input_files": {
        "praise_data": "./praise.csv",
        "rewardboard_list": "./rewardboard.csv"
      },
      "total_tokens_allocated": 1000,
      "payout_token": {
        "token_name": "wxDAI",
        "token_address": "0xe91D153E0b41518A2Ce8Dd3D7944Fa863463a97d"
      },
      "token_reward_percentages": {
        "contributor_rewards": 0.871067,
        "quantifier_rewards": 0.090324,
        "rewardboard_rewards": 0.038608,
        "_comment": "These percentages are tweaked so a regular praise+sourcecred allocation amounts to 7% of total rewards to quantfiers and 3% to the rewardboard."
      },
      "quantification_settings": {
        "number_of_quantifiers_per_praise_receiver": 4,
        "praise_quantify_allowed_values": [
          0, 1, 3, 5, 8, 13, 21, 34, 55, 89, 144
        ],
        "praise_quantify_receiver_prseudonyms": False,
        "praise_quantify_duplicate_praise_valuation": 0.1
      }
}

In [2]:
# Parameters
input_files = {
    "praise_data": "/data/round-1/praise.csv",
    "rewardboard_list": "/data/round-1/rewardboard.csv",
    "cross_period_root": "/data",
}
payout_token = {
    "token_name": "GIV",
    "token_address": "0x4f4F9b8D5B4d0Dc10506e5551B0513B61fD59e75",
}
token_reward_percentages = {
    "contributor_rewards": 0.9,
    "quantifier_rewards": 0.07,
    "rewardboard_rewards": 0.03,
    "ceiling_cutoff": 1500,
}
quantification_settings = {
    "number_of_quantifiers_per_praise_receiver": 4,
    "praise_quantify_allowed_values": [0, 1, 3, 5, 8, 13, 21, 34, 55, 89, 144],
    "praise_quantify_receiver_pseudonyms": False,
    "praise_quantify_duplicate_praise_valuation": 0.1,
}
cross_period_settings = {"cross_period_week_num": 52, "cross_period_step_size": 2}
total_tokens_allocated = "58000"
distribution_name = "round-27"
results_output_folder = "distribution_results"


In [3]:

import os
import sys

import math
import pandas as pd 
import numpy as np 

import scrapbook as sb


In [4]:
PRAISE_DATA_PATH = input_files["praise_data"]
REWARD_BOARD_ADDRESSES_PATH = input_files["rewardboard_list"]

praise_data = pd.read_csv(PRAISE_DATA_PATH)
praise_data.columns = praise_data.columns.str.upper()

rewardboard_addresses = pd.read_csv(REWARD_BOARD_ADDRESSES_PATH)

In [5]:
try:
    CUTOFF_VALUE = token_reward_percentages["ceiling_cutoff"]
except:
    #always distribute total amount of allocated tokens
    CUTOFF_VALUE = 1 

#if the amount of praise is below the cutoff, adapt number of distributed tokens
if(len(praise_data.index) < CUTOFF_VALUE):
        total_tokens_allocated = len(praise_data.index) / CUTOFF_VALUE * float(total_tokens_allocated)



NUMBER_OF_PRAISE_REWARD_TOKENS_TO_DISTRIBUTE = math.trunc( float(total_tokens_allocated) * token_reward_percentages["contributor_rewards"]*1000000) / 1000000
NUMBER_OF_REWARD_TOKENS_FOR_QUANTIFIERS = math.trunc(  float(total_tokens_allocated) * token_reward_percentages["quantifier_rewards"]*1000000) / 1000000
NUMBER_OF_REWARD_TOKENS_FOR_REWARD_BOARD = math.trunc(  float(total_tokens_allocated) * token_reward_percentages["rewardboard_rewards"]*1000000) / 1000000
NUMBER_OF_QUANTIFIERS_PER_PRAISE = quantification_settings["number_of_quantifiers_per_praise_receiver"]





## Reward allocation

### Praise

This method allocates the praise rewards in a very straightforward way: It adds the value of all dished praised together, and then assigns to each user their % of the total.

In [6]:
def calc_praise_rewards(praiseData, tokensToDistribute):

    totalPraisePoints = praiseData['AVG SCORE'].sum()

    praiseData['PERCENTAGE'] = praiseData['AVG SCORE']/totalPraisePoints
    praiseData['TOKEN TO RECEIVE'] = praiseData['PERCENTAGE'] * tokensToDistribute

    return praiseData

praise_distribution = calc_praise_rewards(praise_data.copy(), NUMBER_OF_PRAISE_REWARD_TOKENS_TO_DISTRIBUTE)
praise_distribution.style


,ID,DATE,TO USER ACCOUNT,TO USER ACCOUNT ID,TO ETH ADDRESS,FROM USER ACCOUNT,FROM USER ACCOUNT ID,FROM ETH ADDRESS,REASON,SOURCE ID,SOURCE NAME,SCORE 1,SCORE 2,SCORE 3,DUPLICATE ID 1,DUPLICATE ID 2,DUPLICATE ID 3,DISMISSED 1,DISMISSED 2,DISMISSED 3,QUANTIFIER 1 USERNAME,QUANTIFIER 1 ETH ADDRESS,QUANTIFIER 2 USERNAME,QUANTIFIER 2 ETH ADDRESS,QUANTIFIER 3 USERNAME,QUANTIFIER 3 ETH ADDRESS,AVG SCORE,PERCENTAGE,TOKEN TO RECEIVE
0,635a845c1ab9982803027a71,2021-11-01T23:00:00.000Z,hanners717#2022,6335b56e8622e4e2bd1f1bff,0x711CDABCFdEDEf2c82D9dA533268D8bB6b7468f8,Donna.Ofi,635a84478622e4e2bd2e6ad4,nan,for all her support 😊,DISCORD:679428761438912522:823570903219568700,DISCORD%3AGiveth%3A%E2%9D%A4praise,5,13,0,nan,nan,nan,False,False,False,fuscia,0xA0baFC36DE86d1A64e3f2D01E052A3a2c1487E83,Nicbals#2154,0xd9D5d85B2682830aC58A78c279E0C177953738b4,Ramin#4479,0x10a84b835C5df26f2A380B3E00bCC84A66cD2d34,9.000000,0.001651,19.648502
1,635a845c1ab9982803027a72,2021-11-01T23:00:00.000Z,launchninja,635a84558622e4e2bd2ec21e,nan,karmaticacid#1218,63333aed144cdd9b89d12f9d,0xC46c67Bb7E84490D7EbdD0b8ecDaca68Cf3823F4,for the awesome Giveth / Rainbow rolls marketing and for in general just doing so much to support Giveth!!,DISCORD:679428761438912522:823570903219568700,DISCORD%3AGiveth%3A%E2%9D%A4praise,3,89,0,nan,nan,nan,False,False,False,fuscia,0xA0baFC36DE86d1A64e3f2D01E052A3a2c1487E83,Nicbals#2154,0xd9D5d85B2682830aC58A78c279E0C177953738b4,Ramin#4479,0x10a84b835C5df26f2A380B3E00bCC84A66cD2d34,46.000000,0.008438,100.425679
2,635a845c1ab9982803027a73,2021-11-01T23:00:00.000Z,🌈 🧻chair 🌈 🧻,635a84498622e4e2bd2e7646,nan,karmaticacid#1218,63333aed144cdd9b89d12f9d,0xC46c67Bb7E84490D7EbdD0b8ecDaca68Cf3823F4,for the awesome Giveth / Rainbow rolls marketing and for in general just doing so much to support Giveth!!,DISCORD:679428761438912522:823570903219568700,DISCORD%3AGiveth%3A%E2%9D%A4praise,144,3,55,nan,nan,nan,False,False,False,divine_comedian#5493,0x826976d7C600d45FB8287CA1d7c76FC8eb732030,fuscia,0xA0baFC36DE86d1A64e3f2D01E052A3a2c1487E83,Nicbals#2154,0xd9D5d85B2682830aC58A78c279E0C177953738b4,67.330000,0.012351,146.992629
3,635a845c1ab9982803027a74,2021-11-01T23:00:00.000Z,Amin#2164,63344aa8144cdd9b89d1a234,0xF23eA0b5F14afcbe532A1df273F7B233EBe41C78,karmaticacid#1218,63333aed144cdd9b89d12f9d,0xC46c67Bb7E84490D7EbdD0b8ecDaca68Cf3823F4,"for all their work today on the GIVeconomy, for answering many questions and for pushing GIV to the finish line",DISCORD:679428761438912522:823570903219568700,DISCORD%3AGiveth%3A%E2%9D%A4praise,34,0,55,nan,nan,nan,False,False,False,mateodaza#3156,0x00d18ca9782bE1CaEF611017c2Fbc1a39779A57C,Cori#8483,0xD1Da06878d2817A358498C384418417A752Dfa68,WhyldWanderer#7002,0xed8DB37778804A913670d9367aAf4F043AAd938b,44.500000,0.008163,97.150928
4,635a845c1ab9982803027a75,2021-11-01T23:00:00.000Z,mohammad_ranjbar_z#2709,6334498b144cdd9b89d1a1a2,0x2Ea846Dc38C6b6451909F1E7ff2bF613a96DC1F3,karmaticacid#1218,63333aed144cdd9b89d12f9d,0xC46c67Bb7E84490D7EbdD0b8ecDaca68Cf3823F4,"for all their work today on the GIVeconomy, for answering many questions and for pushing GIV to the finish line",DISCORD:679428761438912522:823570903219568700,DISCORD%3AGiveth%3A%E2%9D%A4praise,8,8,8,nan,nan,nan,False,False,False,toraif#4775,0xe628534EAaFD27ae8C98e02967586fAD4cc70152,karmaticacid#1218,0xC46c67Bb7E84490D7EbdD0b8ecDaca68Cf3823F4,hanners717#2022,0x711CDABCFdEDEf2c82D9dA533268D8bB6b7468f8,8.000000,0.001467,17.465335
5,635a845c1ab9982803027a76,2021-11-01T23:00:00.000Z,Cherik#4711,63333b26144cdd9b89d12fc4,0x8F48094a12c8F99d616AE8F3305D5eC73cBAA6b6,karmaticacid#1218,63333aed144cdd9b89d12f9d,0xC46c67Bb7E84490D7EbdD0b8ecDaca68Cf3823F4,"for all their work today on the GIVeconomy, for answering many questions and for pushing GIV to the finish line",DISCORD:679428761438912522:823570903219568700,DISCORD%3AGiveth%3A%E2%9D%A4praise,89,21,8,nan,nan,nan,False,False,False,geleeroyale#3228,0x701d0ECB3BA780De7b2b36789aEC4493A426010a,Dr. Suga#8514,0x2fEe53687906bA239602eD42CC

### SourceCred
For now Sourcecred does the distribution independently, but if this changed we would calculate the rewards in the same way.

## Preparing and combining the Datasets

Now that we have both distributions, we can combine them into one table.
But before that, we need to prepare the data and clean it a bit. We also use the chance to generate a table which shows us how much praise each user received. We'll use it later in our analysis.

In [7]:

#General Helper func. Puts all the "processing we probably won't need to do later or do differently" in one place
#  -removes the '#' and following from discord names
#  -Some renaming and dropping 
def prepare_praise(praise_data):

    praise_data.rename(columns = {'TO USER ACCOUNT':'USER IDENTITY'}, inplace = True)
    praise_data.rename(columns = {'TO ETH ADDRESS':'USER ADDRESS'}, inplace = True)
    praise_data['USER ADDRESS'].fillna('MISSING USER ADDRESS', inplace=True)
    
    processed_praise = praise_data[['USER IDENTITY', 'USER ADDRESS', 'PERCENTAGE', 'TOKEN TO RECEIVE']]
    praise_by_user = praise_data[['USER IDENTITY', 'USER ADDRESS', 'AVG SCORE', 'PERCENTAGE', 'TOKEN TO RECEIVE']].copy().groupby(['USER IDENTITY', 'USER ADDRESS']).agg('sum').reset_index()
    
    return processed_praise, praise_by_user


processed_praise, praise_by_user = prepare_praise(praise_distribution.copy())
processed_praise.style


,USER IDENTITY,USER ADDRESS,PERCENTAGE,TOKEN TO RECEIVE
0,hanners717#2022,0x711CDABCFdEDEf2c82D9dA533268D8bB6b7468f8,0.001651,19.648502
1,launchninja,MISSING USER ADDRESS,0.008438,100.425679
2,🌈 🧻chair 🌈 🧻,MISSING USER ADDRESS,0.012351,146.992629
3,Amin#2164,0xF23eA0b5F14afcbe532A1df273F7B233EBe41C78,0.008163,97.150928
4,mohammad_ranjbar_z#2709,0x2Ea846Dc38C6b6451909F1E7ff2bF613a96DC1F3,0.001467,17.465335
5,Cherik#4711,0x8F48094a12c8F99d616AE8F3305D5eC73cBAA6b6,0.007214,85.863955
6,"Griff (💜,💜)#8888",0x839395e20bbB182fa440d08F850E6c7A8f6F0780,0.002660,31.655920
7,karmaticacid#1218,0xC46c67Bb7E84490D7EbdD0b8ecDaca68Cf3823F4,0.000917,10.915835
8,"Griff (💜,💜)#8888",0x839395e20bbB182fa440d08F850E6c7A8f6F0780,0.000917,10.915835
9,Danibelle the Uno and Only#6791,0x5D28FE1e9F895464aab52287d85Ebff32B351674,0.000367,4.366334


Let's also create a table which will let us focus on the quantifiers. It will show us what value each quantifier gave to each single praise item.

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [8]:

def data_by_quantifier(praise_data):
    quant_only = pd.DataFrame()
    #praise_data.drop(['DATE', 'TO USER ACCOUNT', 'TO USER ACCOUNT ID', 'TO ETH ADDRESS', 'FROM USER ACCOUNT', 'FROM USER ACCOUNT ID', 'FROM ETH ADDRESS', 'REASON', 'SOURCE ID', 'SOURCE NAME', 'AVG SCORE'], axis=1, inplace=True)
    num_of_quants = NUMBER_OF_QUANTIFIERS_PER_PRAISE
    for i in range(num_of_quants):
        q_name =  str( 'QUANTIFIER '+ str(i+1) +' USERNAME' )
        q_addr =  str( 'QUANTIFIER '+ str(i+1) +' ETH ADDRESS')
        q_value = str('SCORE '+str(i+1) )
        q_duplicate = str('DUPLICATE ID '+str(i+1) )
        
        buf = praise_data[['ID', q_name , q_addr, q_value, q_duplicate]].copy()

        #delete the duplicated rows
        buf = buf.loc[buf[q_duplicate].isnull()] # only include the non-duplicated rows
        buf = buf[['ID', q_name , q_addr, q_value]] # don't need the duplication info anymore
        
    
        buf.rename(columns={q_name: 'QUANT_ID', q_addr: 'QUANT_ADDRESS', q_value: 'QUANT_VALUE', 'ID':'PRAISE_ID'}, inplace=True)

        quant_only = quant_only.append(buf.copy(), ignore_index=True)

    columnsTitles = ['QUANT_ID', 'QUANT_ADDRESS', 'PRAISE_ID', 'QUANT_VALUE']
    quant_only.sort_values(['QUANT_ID', 'PRAISE_ID'], inplace=True)
    quant_only =  quant_only.reindex(columns=columnsTitles).reset_index(drop=True)
    return quant_only

quantifier_rating_table = data_by_quantifier(praise_data.copy())
quantifier_rating_table.style


KeyError: "['QUANTIFIER 4 USERNAME', 'QUANTIFIER 4 ETH ADDRESS', 'SCORE 4', 'DUPLICATE ID 4'] not in index"

Now, we will calculate the rewards for the Quantifiers and the Reward Board. This is fairly straightforward: we distribute the tokens allocated for quantification proportionally to the number of praises quantified, and give all rewardboard members an equal cut.


Before we distribute the rewards, we must remove the quantifiers who didn't show up for this round even though they were drafted

In [ ]:
# clear out the quantifiers who didn't give any rating (i.e. all scores are 0)
quantifier_sum = quantifier_rating_table[['QUANT_ID','QUANT_VALUE']].groupby('QUANT_ID').sum()
norating_quantifiers = quantifier_sum.loc[quantifier_sum['QUANT_VALUE']==0].index.tolist()
norating_quantifiers

In [ ]:


quantifier_rewards = pd.DataFrame(quantifier_rating_table[['QUANT_ID','QUANT_ADDRESS']].value_counts().reset_index().copy())

quantifier_rewards = quantifier_rewards[~quantifier_rewards['QUANT_ID'].isin(norating_quantifiers)]

quantifier_rewards = quantifier_rewards.rename(columns={ quantifier_rewards.columns[2]: "NUMBER_OF_PRAISES" }).reset_index(drop=True)




total_praise_quantified = quantifier_rewards['NUMBER_OF_PRAISES'].sum()
quantifier_rewards['TOKEN TO RECEIVE'] = quantifier_rewards['NUMBER_OF_PRAISES'] / total_praise_quantified  * NUMBER_OF_REWARD_TOKENS_FOR_QUANTIFIERS


    
quantifier_rewards.style

In [ ]:
rewardboard_rewards = pd.DataFrame(rewardboard_addresses)
rewardboard_rewards['TOKEN TO RECEIVE'] = NUMBER_OF_REWARD_TOKENS_FOR_REWARD_BOARD / len(rewardboard_rewards.index)
rewardboard_rewards.style

Now we can merge them all into one table and save it, ready for distribution!

In [ ]:
#def prepare_total_data_chart(praise_rewards, sourcecred_rewards, quantifier_rewards, rewardboard_rewards):
def prepare_total_data_chart(praise_rewards, quantifier_rewards, rewardboard_rewards):
    
    praise_rewards = praise_rewards.copy()[['USER IDENTITY', 'USER ADDRESS', 'TOKEN TO RECEIVE']].rename(columns = {'TOKEN TO RECEIVE':'PRAISE_REWARD'})
    praise_rewards['USER ADDRESS'] = praise_rewards['USER ADDRESS'].str.lower()
    
    quantifier_rewards.rename(columns = {'QUANT_ADDRESS':'USER ADDRESS', 'QUANT_ID': 'USER IDENTITY', 'NUMBER_OF_PRAISES': 'NR_OF_PRAISES_QUANTIFIED', 'TOKEN TO RECEIVE':'QUANT_REWARD'}, inplace = True)
    quantifier_rewards['USER ADDRESS'] = quantifier_rewards['USER ADDRESS'].str.lower()
    
    rewardboard_rewards.rename(columns = {'ID':'USER ADDRESS', 'TOKEN TO RECEIVE': 'REWARDBOARD_REWARD'}, inplace = True)
    rewardboard_rewards['USER ADDRESS'] = rewardboard_rewards['USER ADDRESS'].str.lower()
    
    
    final_allocations = pd.merge(rewardboard_rewards, quantifier_rewards , on=['USER ADDRESS','USER ADDRESS'], how='outer')
    final_allocations = pd.merge(final_allocations, praise_rewards, left_on=['USER ADDRESS'], right_on=['USER ADDRESS'], how='outer')
    
    #now we can merge the IDs, replacing any missing values
    final_allocations['USER IDENTITY_x']= final_allocations['USER IDENTITY_x'].combine_first(final_allocations['USER IDENTITY_y'])
    final_allocations.rename(columns = {'USER IDENTITY_x': 'USER IDENTITY'},  inplace = True)
    final_allocations.drop('USER IDENTITY_y', axis=1, inplace=True)
    
    
    final_allocations['USER IDENTITY'].fillna('missing username', inplace = True)
    final_allocations.fillna(0, inplace = True)
    final_allocations['TOTAL TO RECEIVE'] = final_allocations['PRAISE_REWARD'] + final_allocations['QUANT_REWARD'] + final_allocations['REWARDBOARD_REWARD']
   
    
    final_allocations = final_allocations.sort_values(by= 'TOTAL TO RECEIVE', ascending  = False).reset_index(drop=True)
    
    #put the columns into the desired order
    final_allocations = final_allocations[['USER IDENTITY', 'USER ADDRESS', 'PRAISE_REWARD', 'QUANT_REWARD','NR_OF_PRAISES_QUANTIFIED', 'REWARDBOARD_REWARD', 'TOTAL TO RECEIVE']]
    
    
    return final_allocations


final_token_allocations = prepare_total_data_chart(praise_by_user.copy(), quantifier_rewards.copy(), rewardboard_rewards.copy())
final_token_allocations.style


### "Glue" relevant DataFrames to send to analysis

In [ ]:
sb.glue("final_token_allocations", final_token_allocations, 'pandas')
sb.glue("rewardboard_rewards", rewardboard_rewards, 'pandas')
sb.glue("quantifier_rewards", quantifier_rewards, 'pandas')
sb.glue("quantifier_rating_table", quantifier_rating_table, 'pandas')

sb.glue("processed_praise", processed_praise, 'pandas')
sb.glue("praise_by_user", praise_by_user, 'pandas')


sb.glue("praise_distribution", praise_distribution, 'pandas')
sb.glue("quantifiers_per_praise", quantification_settings["number_of_quantifiers_per_praise_receiver"])
sb.glue("distribution_name", distribution_name)
sb.glue("total_tokens_allocated", total_tokens_allocated)
sb.glue("praise_quantify_duplicate_praise_valuation", quantification_settings['praise_quantify_duplicate_praise_valuation'])
sb.glue("pseudonyms_used", quantification_settings['praise_quantify_receiver_pseudonyms'])

### Save the distribution files

In [ ]:
final_allocation_csv = final_token_allocations.to_csv(index=False)
with open('final_praise_token_allocation.csv', 'w') as f:
    f.write(final_allocation_csv)

In [ ]:
#create "transactions" dist
final_alloc_aragon = final_token_allocations[['USER ADDRESS', 'TOTAL TO RECEIVE']].copy()
final_alloc_aragon['TOKEN SYMBOL'] = payout_token['token_name']
final_alloc_aragon = final_alloc_aragon[final_alloc_aragon['USER ADDRESS'] != "missing user address"]
final_alloc_aragon = final_alloc_aragon.to_csv(sep=',', index=False, header=False)
with open('praise_aragon_distribution.csv', 'w') as f:
    f.write(final_alloc_aragon)

In [ ]:
praise_reward_export = praise_distribution.to_csv(index=False)
with open('extended_praise_data.csv', 'w') as f:
    f.write(praise_reward_export)